In [28]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from datasets import load_dataset, Dataset, load_metric
from torch.utils.data import DataLoader
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
import torch
import numpy as np

os.getcwd()
#os.chdir('c:\\Users\\Kirchner\\Desktop\\Masterarbeit')

ModuleNotFoundError: No module named 'tensorflow'

In [3]:

import gc

gc.collect()

torch.cuda.empty_cache()

In [3]:
# train_size = 50
# def read_annotations(txt_name, mode):
#     texts = []
#     labels = []

#     with open (txt_name, encoding = "utf-8", mode = "r+") as f:
#         for line in f.readlines():
#             texts.append(line.split("\t")[8])
#             if mode == "opinion":
#                 labels.append(line.split("\t")[5])
#             elif mode == "sentiment":
#                 labels.append(line.split("\t")[6])
#             else:
#                 print(f"There is no label for {mode}")

#     return texts,labels

In [3]:
# meine_texte, meine_labels  = read_annotations("cleaned_annotated_data_training.txt", "sentiment")

# texts_train = meine_texte[:train_size]
# texts_test = meine_texte[train_size:]
# labels_train = meine_labels[:train_size]
# labels_test = meine_labels[train_size:]

# print(len(labels_train), len(labels_test))

In [4]:
### Here we import the tokenizer that we already trained if we change the flag to False, otherwise we take the default
### model
default = True
if os.path.isdir("meine_tokenizer") and not default:
    print("Loading finetuned tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained("meine_tokenizer_10E_sentiment_500")
else:
    tokenizer = AutoTokenizer.from_pretrained("oliverguhr/german-sentiment-bert")

if os.path.isdir("Sentiment_Bert_mit_spiegel") and not default:
    print("Loading finetuned model...")
    model = AutoModelForSequenceClassification.from_pretrained("Sentiment_Bert_mit_spiegel_10E_sentiment_500")
else:
    model = AutoModelForSequenceClassification.from_pretrained("oliverguhr/german-sentiment-bert")

In [12]:
import pandas as pd
import torch
from IPython.display import display
from scipy.stats import entropy

In [35]:
data_text = []
data_labels = []
text_train = []
labels_train = []
text_test = np.array([])
labels_test = np.array([])


label_to_id = {"positive" : 0, "negative" : 1, "neutral" : 2}
id_to_label = {0:"positive", 1:"negative", 2:"neutral"}

### Here we create a training set that can be used to compare across different sizes of training data

mein_dataframe1 = pd.read_csv("annotated_data_with_users_automatic.csv", header=None)
mein_dataframe1.columns = ["ID", "Date", "Time", "Comment Level", "Username", "Opinion", "Sentiment",
                          "topic_comment", "Topic_article", "Comment", "Method"]

display(len(mein_dataframe1))
# split_index = int(len(mein_dataframe1)*0.8)
for index, row in mein_dataframe1.iterrows():
    data_text.append(row["Comment"])
    data_labels.append(row["Sentiment"])

print(data_labels)   
# ### Define test and train set
# text_test = text_train[split_index+1:]
# text_train = text_train[:split_index]
# labels_test = labels_train[split_index+1:]
# labels_train = labels_train[:split_index]


### Replace label as int
for idx, labels in enumerate(data_labels):
    data_labels[idx] = label_to_id[labels]


# for idx, labels in enumerate(labels_test):
#     labels_test[idx] = label_to_id[labels]
    
    

# with open ("cleaned_annotated_data_training.txt", encoding="utf-8", mode="r+") as f:
#     training_data = f.readlines()
#     split_index = int(len(mein_dataframe1)*0.8)
    
#     ### The split on index 6 gives us Sentiment, split on 5 Opinion
    
#     ### The first 88 annotations had been annotated without a article topic
#     for line in training_data[:88]:
#         text_train.append(line.split("\t")[8])
#         labels_train.append(label_to_id[line.split("\t")[6]])
#     for line in training_data[89:split_index]:
#         text_train.append(line.split("\t")[9])
#         labels_train.append(label_to_id[line.split("\t")[6]])
#     for line in training_data[split_index+1:]:
#         text_test.append(line.split("\t")[9])
#         labels_test.append(label_to_id[line.split("\t")[6]])

def preprocess_function(examples):
    return tokenizer(examples["texts"], padding = True, truncation=True)

def calculate_entropy(logits):
    probas = torch.nn.Softmax(dim=1)(torch.from_numpy(logits))
    samples_entropy = entropy(probas.transpose(0, 1).cpu())
    samples_entropy = torch.from_numpy(samples_entropy)
    return samples_entropy

def compute_metrics_accuracy(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        print("Predictions", predictions, "Labels", labels)
        return metric.compute(predictions=predictions, references=labels)

### Define splits for k fold cross validation
folds = StratifiedKFold(n_splits=5)

### Convert to numpy array
data_text = np.array([i for i in data_text])
data_labels = np.array([i for i in data_labels])



accuracies = []
metric = load_metric("accuracy")
hello = []
for train_index, test_index in folds.split(data_text, data_labels):
    text_train, labels_train  = data_text[train_index], data_labels[train_index]
    text_test, labels_test  = data_text[test_index], data_labels[test_index]

    train_dict = {"texts": text_train, "labels" : labels_train}
    test_dict = {"texts": text_test, "labels" : labels_test}

    # print([len(z) for z in [text_train, text_test]])


    ### Remove newline characters
    for idx, text in enumerate(train_dict["texts"]):
        train_dict["texts"][idx] = text.rstrip("\n")

    for idx2, text2 in enumerate(test_dict["texts"]):
        test_dict["texts"][idx2] = text2.rstrip("\n")
        
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    
    meine_dataset_train = Dataset.from_dict(train_dict)

    mein_dataset_test = Dataset.from_dict(test_dict)

    tokenized_dataset_train = meine_dataset_train.map(preprocess_function, batched= True)
    tokenized_dataset_test = mein_dataset_test.map(preprocess_function, batched=True)

    training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps=20,
    evaluation_strategy="epoch"
    )


    trainer = Trainer(
        model= AutoModelForSequenceClassification.from_pretrained("oliverguhr/german-sentiment-bert"),
        args=training_args,
        train_dataset=tokenized_dataset_train,
        eval_dataset=tokenized_dataset_test,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics = compute_metrics_accuracy
    )

    trainer.train()
#     index_topk = torch.topk(calculate_entropy(trainer.predict(tokenized_dataset_test).predictions), 20).indices
#     print(index_topk)
#     for x in index_topk:
#         print(meine_dataset_train["texts"][x], "SATZ")
    accuracies.append(trainer.evaluate()['eval_accuracy'])
    

1523

['negative', 'negative', 'positive', 'neutral', 'negative', 'negative', 'negative', 'neutral', 'negative', 'negative', 'negative', 'negative', 'neutral', 'negative', 'neutral', 'neutral', 'negative', 'neutral', 'negative', 'negative', 'neutral', 'neutral', 'neutral', 'negative', 'negative', 'neutral', 'negative', 'negative', 'negative', 'neutral', 'negative', 'neutral', 'negative', 'negative', 'neutral', 'neutral', 'negative', 'negative', 'neutral', 'neutral', 'neutral', 'neutral', 'negative', 'negative', 'positive', 'neutral', 'neutral', 'negative', 'neutral', 'negative', 'neutral', 'neutral', 'neutral', 'negative', 'neutral', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'neutral', 'negative', 'negative', 'negative', 'neutral', 'neutral', 'neutral', 'neutral', 'negative', 'neutral', 'negative', 'negative', 'neutral', 'negative', 'neutral', 'negative', 'negative', 'negative', 'negative', 'negative', 'neutral', 'negative', 'negative', 'neutral', 'positive', 'n

W&B installed but not logged in. Run `wandb login` or set the WANDB_API_KEY env variable.
/local/username/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.632600,0.707747,0.639344,9.786900,31.164000
2,0.568300,0.769817,0.668852,9.790200,31.154000
3,0.382000,1.031478,0.655738,9.831000,31.024000
4,0.322900,1.484996,0.675410,9.823500,31.048000
5,0.387800,1.884604,0.659016,9.893600,30.828000
6,0.236300,1.986675,0.672131,9.813900,31.078000
7,0.302400,2.206053,0.681967,9.803400,31.112000
8,0.092000,2.267210,0.678689,9.803700,31.111000
9,0.104000,2.320161,0.672131,9.798400,31.127000
10,0.104300,2.387798,0.672131,9.887400,30.847000


Predictions [1 1 2 1 2 1 1 2 1 1 1 1 1 2 1 2 1 1 1 1 2 2 1 2 1 2 1 1 1 2 1 2 2 1 2 2 1
 1 2 1 2 2 2 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1 2
 1 2 1 1 1 1 1 2 1 2 1 2 1 1 2 1 1 1 1 1 2 1 2 1 1 1 1 2 1 2 1 1 1 1 2 1 2
 1 1 1 1 2 1 1 1 1 1 1 2 2 1 2 1 1 1 1 1 2 1 1 1 1 2 1 1 1 1 1 2 1 1 1 1 1
 2 1 2 2 1 2 1 2 1 1 1 2 2 2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 2 1 2 1 2 1 1 1 1
 1 1 1 2 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 2 1 1 2 2 1 2 1 1 1 1 2 1 1
 1 2 1 1 2 1 1 2 1 2 2 1 1 1 2 1 2 1 2 2 1 1 1 1 2 1 1 1 1 2 2 1 1 1 2 2 1
 1 1 1 1 1 1 1 2 1 1 1 2 1 2 1 1 1 1 1 1 2 2 1 1 1 2 2 1 1 0 1 1 2 2 2 1 1
 1 2 2 2 1 1 1 1 2] Labels [1 1 0 2 1 1 1 2 1 1 1 1 2 1 2 2 1 2 1 1 2 2 2 1 1 2 1 1 1 2 1 2 1 1 2 2 1
 1 2 2 2 2 1 1 0 2 2 1 2 1 2 2 2 1 2 1 1 1 1 1 1 2 1 1 1 2 2 2 2 1 2 1 1 2
 1 2 1 1 1 1 1 2 1 1 2 0 1 1 1 2 1 1 2 1 0 2 2 2 1 1 1 2 1 2 1 1 1 1 2 2 1
 1 1 2 1 2 1 1 2 1 1 1 1 2 1 2 1 1 2 1 1 2 2 1 2 1 2 1 1 2 2 1 1 2 1 1 1 1
 2 2 1 2 2 1 1 2 1 2 1 2 1 1 1 0 1 1 2 1 1 2 2 1 1 2 1 1 2 2 

...

/local/username/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Predictions [2 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 2 2 1 1 1 1 2 1 1 2 2 1 2 1 2 1
 1 1 1 2 2 2 2 1 2 2 1 1 2 2 1 1 1 2 2 1 1 2 2 1 1 1 2 1 2 1 2 2 2 1 1 2 2
 1 2 1 2 2 2 2 1 2 1 2 1 1 1 1 1 2 2 1 1 2 1 1 1 1 1 1 2 1 2 2 2 2 1 1 1 1
 1 1 1 2 1 1 2 0 2 1 2 1 1 2 2 0 2 1 2 1 1 1 0 2 2 1 1 1 2 2 2 2 2 2 2 0 2
 1 2 1 2 2 1 1 1 1 1 1 1 1 2 1 2 1 2 2 1 2 2 1 2 1 1 2 2 2 1 1 1 2 2 2 2 1
 2 2 1 1 2 2 2 1 2 2 1 2 1 2 2 2 2 1 1 2 2 1 1 2 2 1 1 2 2 1 1 1 1 1 2 2 1
 2 1 1 2 2 2 2 1 1 2 2 2 2 1 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2 1 2 1 1 1 2
 2 1 2 1 2 2 1 1 2 2 2 1 2 1 2 2 2 1 1 0 1 1 1 1 2 2 0 1 1 1 1 2 1 1 2 1 1
 2 1 2 1 2 2 2 2] Labels [0 1 1 1 1 1 1 1 2 2 1 1 2 1 1 1 2 2 2 2 1 2 2 1 1 1 1 1 2 2 2 2 1 2 2 1 1
 2 2 1 1 1 1 2 1 1 2 1 1 1 1 1 2 1 2 1 1 1 2 2 2 2 1 1 1 1 1 1 2 2 1 1 2 1
 1 2 1 1 1 2 1 2 1 1 2 1 1 2 1 1 1 2 2 0 2 1 1 2 1 1 1 2 1 2 1 2 2 1 2 1 0
 2 1 1 2 2 1 2 2 2 1 2 1 1 1 2 1 2 1 1 2 1 1 2 1 1 1 1 1 1 2 1 2 1 1 2 2 1
 2 1 1 1 2 1 2 1 1 1 1 1 1 1 2 2 0 2 2 1 2 2 1 1 1 2 2 2 2 1 1 

/local/username/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Predictions [2 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 2 2 1 1 1 1 2 1 1 2 2 1 2 1 2 1
 1 1 1 2 2 2 2 1 2 2 1 1 2 2 1 1 1 2 2 1 1 2 2 1 1 1 2 1 2 1 2 2 2 1 2 2 2
 1 2 1 2 2 2 2 1 2 1 2 1 1 1 1 1 2 2 1 1 2 1 1 1 1 1 1 2 1 2 2 2 2 1 1 1 2
 1 1 1 2 1 1 2 0 2 1 2 1 1 2 2 0 2 1 2 1 1 1 0 2 2 1 1 2 2 2 2 2 2 2 2 0 2
 1 2 0 2 2 1 1 1 1 1 1 1 1 2 1 2 1 2 2 1 2 2 1 2 1 1 2 2 2 1 1 1 2 2 2 2 1
 2 2 1 1 2 2 2 1 2 2 1 2 1 2 2 2 2 1 1 2 2 1 1 2 2 1 1 2 2 1 1 1 1 1 2 2 1
 2 1 1 2 2 2 2 1 1 2 2 2 2 1 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2 1 2 1 1 1 2
 2 1 2 1 2 2 1 1 2 2 2 1 2 1 2 2 2 1 1 0 1 2 1 1 2 2 0 1 1 1 1 2 1 1 2 1 1
 2 1 2 1 2 2 2 2] Labels [0 1 1 1 1 1 1 1 2 2 1 1 2 1 1 1 2 2 2 2 1 2 2 1 1 1 1 1 2 2 2 2 1 2 2 1 1
 2 2 1 1 1 1 2 1 1 2 1 1 1 1 1 2 1 2 1 1 1 2 2 2 2 1 1 1 1 1 1 2 2 1 1 2 1
 1 2 1 1 1 2 1 2 1 1 2 1 1 2 1 1 1 2 2 0 2 1 1 2 1 1 1 2 1 2 1 2 2 1 2 1 0
 2 1 1 2 2 1 2 2 2 1 2 1 1 1 2 1 2 1 1 2 1 1 2 1 1 1 1 1 1 2 1 2 1 1 2 2 1
 2 1 1 1 2 1 2 1 1 1 1 1 1 1 2 2 0 2 2 1 2 2 1 1 1 2 2 2 2 1 1 

Predictions [2 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1 2 1 1 1 1 2 2 1 1 1 1 2 1 1 2 2 1 2 1 2 1
 1 1 1 2 2 2 2 1 2 2 1 1 2 2 1 1 1 2 2 1 1 2 2 1 1 1 2 1 2 1 2 2 2 1 2 2 2
 1 2 1 2 2 2 2 1 2 1 2 1 1 1 1 1 2 2 1 1 2 1 1 1 1 1 1 2 1 2 2 2 2 1 1 1 2
 1 1 1 2 1 1 2 0 2 1 2 1 1 2 2 0 2 1 2 1 1 1 0 2 2 1 1 2 2 2 2 2 2 2 2 0 2
 1 2 0 2 2 1 1 1 1 1 1 1 1 2 1 2 1 2 2 1 2 2 1 2 1 1 2 2 2 1 1 1 2 2 2 2 1
 2 2 1 1 2 2 2 1 2 2 1 2 1 2 2 2 2 1 1 2 2 1 1 2 2 1 1 2 2 1 1 1 1 1 2 2 1
 2 1 1 2 2 2 2 1 1 2 2 2 2 1 2 1 2 1 2 1 1 2 1 1 2 1 2 1 2 2 2 1 2 1 1 1 2
 2 1 2 1 2 2 1 1 2 2 2 1 2 1 2 2 2 1 1 0 1 2 1 1 2 2 0 1 1 1 1 2 1 1 2 1 1
 2 1 2 1 2 2 2 2] Labels [0 1 1 1 1 1 1 1 2 2 1 1 2 1 1 1 2 2 2 2 1 2 2 1 1 1 1 1 2 2 2 2 1 2 2 1 1
 2 2 1 1 1 1 2 1 1 2 1 1 1 1 1 2 1 2 1 1 1 2 2 2 2 1 1 1 1 1 1 2 2 1 1 2 1
 1 2 1 1 1 2 1 2 1 1 2 1 1 2 1 1 1 2 2 0 2 1 1 2 1 1 1 2 1 2 1 2 2 1 2 1 0
 2 1 1 2 2 1 2 2 2 1 2 1 1 1 2 1 2 1 1 2 1 1 2 1 1 1 1 1 1 2 1 2 1 1 2 2 1
 2 1 1 1 2 1 2 1 1 1 1 1 1 1 2 2 0 2 2 1 2 2 1 1 1 2 2 2 2 1 1 

In [36]:
### Cross validated accuracy
print(len(text_train))
sum(accuracies)/len(accuracies)





1219


0.6401488352027609

In [71]:
# train_text_tokenized = tokenizer(text_train, truncation=True, padding=True)

# test_text_tokenized = tokenizer(text_test, truncation=True, padding=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [72]:
def preprocess_function(examples):
    return tokenizer(examples["texts"], padding = True, truncation=True)


In [25]:
# spiegel = SpiegelDataset("cleaned_annotated_data_training.txt")

# spiegel_data = load_dataset("text", data_files={"train": "Spiegel_train.txt", "test" : "Spiegel_test.txt"})

Using custom data configuration default-b87b2cbca73061b8
Reusing dataset text (C:\Users\Kirchner\.cache\huggingface\datasets\text\default-b87b2cbca73061b8\0.0.0\e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5)
100%|██████████| 2/2 [00:00<00:00, 795.81it/s]


In [73]:

test_dict = {"texts" : text_test,
            "labels" : labels_test}
            

metric = load_metric("accuracy")

def compute_metrics_accuracy(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    print("Predictions", predictions, "Labels", labels)
    return metric.compute(predictions=predictions, references=labels)


meine_dataset_train = Dataset.from_dict(train_dict)

mein_dataset_test = Dataset.from_dict(test_dict)


tokenized_dataset_train = meine_dataset_train.map(preprocess_function, batched= True)
tokenized_dataset_test = mein_dataset_test.map(preprocess_function, batched=True)

In [74]:
for x in tokenized_dataset_test["texts"]:
    print(len(x))


176
158
514
149
285
228
379
197
373
242
399
547
252
669
105
601
944
301
353
329
216
315
149
244
423
589
369
434
392
594
137
457
417
147
298
117
561
62
389
312
180
661
160
173
132
85
650
174
398
703
891
454
120
220
1074
159
238
534
451
431
181
308
117
146
84
468
100
306
193
167
554
79
771
409
846
403
350
477
193
296
635
274
201
191
347
124
796
173
550
109
318
211
81
537
781
459
513
106
374
156
312
156
168
310
97
250
131
93
323
136
214
339
188
545
391
375
313
691
424
333
510
253
480
178
126
203
130
270
436
376
360
263
539
547
174
222
254
226
328
660
263
238
562
70
821
328
378
702
190
509
294
449
409
151
406
675
345
414
313
297
233
101
275
428
494
466
70
510
1376
326
287
455
241
983
345
278
292
264
138
135
105
236
468
359
364
433
300
741
352
137
124
197
929
188
844
206
181
132
144
91
207
104
439
289
306
104
873
459
397
252
576
471
121
386
257
97
118
234
444
648
572
281
288
127


In [75]:
training_args = TrainingArguments(
    output_dir='./results',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_steps=20,
    evaluation_strategy="epoch"
    
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics = compute_metrics_accuracy
)

trainer.train()
# trainer.evaluate()


W&B installed but not logged in. Run `wandb login` or set the WANDB_API_KEY env variable.
/local/username/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,0.787600,0.751194,0.642857,12.594500,17.786000
2,0.526500,0.883527,0.602679,10.334500,21.675000
3,0.380900,1.501034,0.669643,10.227100,21.903000
4,0.398300,2.193896,0.602679,10.247200,21.860000
5,0.151100,2.759660,0.616071,10.233900,21.888000
6,0.075300,3.217661,0.593750,12.104400,18.506000
7,0.005500,3.350807,0.607143,11.324300,19.781000
8,0.063700,3.392122,0.620536,10.154500,22.059000
9,0.000400,3.372744,0.616071,10.282700,21.784000
10,0.046200,3.416090,0.620536,10.338800,21.666000


Predictions [2 1 2 2 1 1 1 2 1 1 1 1 1 2 1 1 2 1 1 1 1 1 1 1 2 1 1 2 1 1 2 1 1 2 1 0 1
 2 1 0 1 1 2 2 2 1 1 2 1 1 1 2 2 1 1 2 2 1 2 1 1 1 2 2 1 1 1 1 2 1 1 1 1 1
 1 1 1 1 1 1 0 2 1 1 1 1 1 1 1 1 1 2 2 1 1 1 2 1 1 1 1 1 1 1 2 2 2 2 1 2 1
 2 2 1 1 1 1 1 1 1 1 2 1 2 1 2 2 2 1 2 1 2 1 1 1 0 1 1 2 1 1 1 2 1 1 1 1 1
 1 1 1 2 1 2 1 1 2 1 1 2 1 2 2 1 1 1 1 1 1 2 1 2 2 1 2 1 1 1 2 1 2 1 1 1 1
 2 1 1 2 1 1 2 2 1 1 1 1 1 2 1 1 2 1 1 1 1 2 1 1 1 2 1 2 1 1 2 1 1 1 1 1 1
 1 2] Labels [1 1 1 2 2 1 1 2 2 1 1 0 1 2 2 1 2 2 1 2 1 1 2 1 2 2 1 2 2 2 2 1 1 2 1 1 1
 0 1 2 1 1 1 2 2 1 2 1 2 1 2 1 2 1 2 2 2 1 2 2 2 1 2 1 1 1 1 1 2 1 1 1 1 1
 1 1 1 2 0 1 0 2 1 1 2 1 1 2 2 1 2 2 2 2 1 1 0 1 1 1 1 1 1 1 1 0 2 1 1 2 2
 1 2 2 1 1 1 1 2 1 1 2 1 2 2 1 2 2 1 1 2 1 2 1 1 1 2 1 2 1 2 1 2 1 1 2 1 1
 2 1 1 1 2 2 2 2 2 1 1 1 1 2 1 1 1 2 2 2 1 1 1 1 2 1 0 1 2 2 2 1 1 2 1 1 2
 2 1 2 2 1 1 2 2 1 1 1 1 1 2 1 2 1 1 1 1 2 2 1 1 2 2 1 2 2 2 2 2 1 2 2 2 1
 1 1]
Predictions [2 1 2 2 2 2 2 2 1 1 2 2 1 2 1 1 2 2 2 1 2 1 2 1 2 1 1 2 

/local/username/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Predictions [2 1 2 2 0 2 1 2 1 1 2 0 2 2 1 1 2 1 0 1 1 1 2 1 2 1 1 2 1 2 2 1 2 2 1 2 1
 2 1 0 1 1 2 2 2 1 2 2 1 1 1 2 2 1 2 2 2 1 2 2 1 1 2 2 1 1 1 1 2 1 1 1 2 2
 2 1 1 2 1 2 0 2 1 1 0 1 1 1 2 1 1 2 2 1 1 1 1 1 1 1 1 1 1 0 2 0 2 2 1 2 1
 2 2 2 1 1 1 1 1 1 1 2 1 2 1 2 2 2 1 2 2 2 2 1 1 0 1 1 2 1 2 2 2 2 2 1 2 1
 1 1 1 2 1 2 2 2 2 1 1 2 1 2 2 1 1 2 1 1 1 2 1 2 2 2 2 1 1 1 2 1 2 1 1 1 1
 2 0 1 2 2 1 2 2 1 2 1 1 1 2 1 1 2 2 2 1 1 2 2 1 1 2 1 2 2 1 1 2 2 1 1 2 2
 2 2] Labels [1 1 1 2 2 1 1 2 2 1 1 0 1 2 2 1 2 2 1 2 1 1 2 1 2 2 1 2 2 2 2 1 1 2 1 1 1
 0 1 2 1 1 1 2 2 1 2 1 2 1 2 1 2 1 2 2 2 1 2 2 2 1 2 1 1 1 1 1 2 1 1 1 1 1
 1 1 1 2 0 1 0 2 1 1 2 1 1 2 2 1 2 2 2 2 1 1 0 1 1 1 1 1 1 1 1 0 2 1 1 2 2
 1 2 2 1 1 1 1 2 1 1 2 1 2 2 1 2 2 1 1 2 1 2 1 1 1 2 1 2 1 2 1 2 1 1 2 1 1
 2 1 1 1 2 2 2 2 2 1 1 1 1 2 1 1 1 2 2 2 1 1 1 1 2 1 0 1 2 2 2 1 1 2 1 1 2
 2 1 2 2 1 1 2 2 1 1 1 1 1 2 1 2 1 1 1 1 2 2 1 1 2 2 1 2 2 2 2 2 1 2 2 2 1
 1 1]
Predictions [2 1 2 2 2 2 2 2 1 1 2 2 2 2 1 1 2 2 2 1 1 1 2 1 2 1 1 2 

/local/username/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Predictions [2 1 2 2 0 2 1 2 1 1 2 0 2 2 1 1 2 1 2 1 1 1 2 1 2 1 1 2 1 2 2 1 2 2 1 2 1
 2 1 0 1 1 2 2 2 1 2 2 1 1 1 2 2 1 2 2 2 1 2 2 1 1 2 2 1 1 1 1 2 1 1 1 2 2
 2 1 1 2 1 2 0 2 1 1 2 1 1 1 2 1 1 2 2 1 2 1 1 1 1 1 2 1 1 0 2 2 2 2 1 2 1
 2 2 2 1 1 1 1 1 1 1 2 1 2 1 2 2 2 1 2 2 2 2 1 1 0 1 1 2 1 2 2 2 2 2 1 2 1
 2 1 1 2 1 2 2 2 2 1 1 2 1 2 2 1 1 2 1 1 1 2 1 2 2 2 2 1 1 1 2 1 2 1 1 1 1
 2 0 1 2 1 1 2 1 1 2 1 1 1 2 1 1 2 2 2 1 1 2 2 1 1 2 1 2 2 1 1 2 1 1 1 2 2
 2 2] Labels [1 1 1 2 2 1 1 2 2 1 1 0 1 2 2 1 2 2 1 2 1 1 2 1 2 2 1 2 2 2 2 1 1 2 1 1 1
 0 1 2 1 1 1 2 2 1 2 1 2 1 2 1 2 1 2 2 2 1 2 2 2 1 2 1 1 1 1 1 2 1 1 1 1 1
 1 1 1 2 0 1 0 2 1 1 2 1 1 2 2 1 2 2 2 2 1 1 0 1 1 1 1 1 1 1 1 0 2 1 1 2 2
 1 2 2 1 1 1 1 2 1 1 2 1 2 2 1 2 2 1 1 2 1 2 1 1 1 2 1 2 1 2 1 2 1 1 2 1 1
 2 1 1 1 2 2 2 2 2 1 1 1 1 2 1 1 1 2 2 2 1 1 1 1 2 1 0 1 2 2 2 1 1 2 1 1 2
 2 1 2 2 1 1 2 2 1 1 1 1 1 2 1 2 1 1 1 1 2 2 1 1 2 2 1 2 2 2 2 2 1 2 2 2 1
 1 1]
Predictions [2 1 2 2 0 2 1 2 1 1 2 0 2 2 1 1 2 1 2 1 1 1 2 1 2 1 1 2 

TrainOutput(global_step=1130, training_loss=0.25791671365639607, metrics={'train_runtime': 913.8611, 'train_samples_per_second': 1.237, 'total_flos': 2339215992120240, 'epoch': 10.0})

In [12]:
torch.cuda.get_device_name(0)

'GeForce GTX TITAN X'

In [19]:

model.save_pretrained("Sentiment_Bert_mit_spiegel_10E_sentiment_1000_2e-5")
tokenizer.save_pretrained("meine_tokenizer_10E_sentiment_1000_2e-5")


('meine_tokenizer_10E_sentiment_1000_2e-5/tokenizer_config.json',
 'meine_tokenizer_10E_sentiment_1000_2e-5/special_tokens_map.json',
 'meine_tokenizer_10E_sentiment_1000_2e-5/vocab.txt',
 'meine_tokenizer_10E_sentiment_1000_2e-5/added_tokens.json')

In [32]:
test_dict["labels"]

[1,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 0,
 1,
 1,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 2]

In [68]:
len(train_text_tokenized["attention_mask"])
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [143]:


# training_args = TrainingArguments(output_dir="./results",
#                     learning_rate=2e-5,
#                     per_device_eval_batch_size=4,
#                     per_device_train_batch_size=4,
#                     num_train_epochs=3,
#                     weight_decay=0.01
#                     )

# trainer = Trainer(
#             model=model,
#             args=training_args,
#             train_dataset=train_text_tokenized,
#             eval_dataset=test_text_tokenized,
#             tokenizer = tokenizer,
#             data_collator= data_collator
# )


# trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 3
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 3





AttributeError: 'list' object has no attribute 'keys'

In [5]:
from germansentiment import SentimentModel

['Nachtrag: Dieser Artikel ist wirklich enttäuschend. Dass es manchem Leser vielleicht schwerfällt, sich an den Ablauf der Ereignisse zu erinnern, oder diese in einen Kontext zu setzen, okay. Aber der Spiegel? Da könnte man ja wirklich vermuten, das sei Click-Baiting, um Seitenaufrufe zu generieren, oder vom US-Außenministerium abgeschrieben. Wahrscheinlich aber doch einfach nur Naivität und leichte Beeinflussbarkeit der Autoren.\n',
 'Ein Flugzeug aus China macht noch keinen Frühling. Die Kritik dieses Artikels ist völlig unangebracht. Die EU hat gesundheitspolitisch fast keine Befugnisse. Das ist so gewollt und an sich auch sehr vorteilhaft. Koordinierte politische Aktionen auf europäischer Ebene würden hier gar nicht weiterhelfen. Jede Region in Europa trifft die für sie und der Lage angemessenen Entscheidungen. Das gibt dann kein einheitliches Bild ab, aber der Virus tritt auch nicht überall einheitlich auf. Die einzige Ebene auf der ein Austausch wirklich erforderlich ist, ist die

In [24]:
model = SentimentModel()
all = 0
correct = 0

hello = [id_to_label[x] for x in train_dict["labels"][:150]]

for idx, word in enumerate(model.predict_sentiment(train_dict["text"][:150])):
    print(word, idx)
    all += 1
    if hello[idx] == word:
        correct+= 1

print(correct/all)


RuntimeError: CUDA out of memory. Tried to allocate 800.00 MiB (GPU 0; 4.00 GiB total capacity; 1.90 GiB already allocated; 317.91 MiB free; 2.53 GiB reserved in total by PyTorch)

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_imdb = imdb.map(preprocess_function, batched=True)